# Contextual Compression in Document Retrieval

## Overview

This code demonstrates the implementation of contextual compression in a document retrieval system using LangChain and OpenAI's language models. The technique aims to improve the relevance and conciseness of retrieved information by compressing and extracting the most pertinent parts of documents in the context of a given query.

## Motivation

Traditional document retrieval systems often return entire chunks or documents, which may contain irrelevant information. Contextual compression addresses this by intelligently extracting and compressing only the most relevant parts of retrieved documents, leading to more focused and efficient information retrieval.

## Key Components

1. Vector store creation from a PDF document
2. Base retriever setup
3. LLM-based contextual compressor
4. Contextual compression retriever
5. Question-answering chain integrating the compressed retriever

## Method Details

### Document Preprocessing and Vector Store Creation

1. The PDF is processed and encoded into a vector store using a custom `encode_pdf` function.

### Retriever and Compressor Setup

1. A base retriever is created from the vector store.
2. An LLM-based contextual compressor (LLMChainExtractor) is initialized using OpenAI's GPT-4 model.

### Contextual Compression Retriever

1. The base retriever and compressor are combined into a ContextualCompressionRetriever.
2. This retriever first fetches documents using the base retriever, then applies the compressor to extract the most relevant information.

### Question-Answering Chain

1. A RetrievalQA chain is created, integrating the compression retriever.
2. This chain uses the compressed and extracted information to generate answers to queries.

## Benefits of this Approach

1. Improved relevance: The system returns only the most pertinent information to the query.
2. Increased efficiency: By compressing and extracting relevant parts, it reduces the amount of text the LLM needs to process.
3. Enhanced context understanding: The LLM-based compressor can understand the context of the query and extract information accordingly.
4. Flexibility: The system can be easily adapted to different types of documents and queries.

## Conclusion

Contextual compression in document retrieval offers a powerful way to enhance the quality and efficiency of information retrieval systems. By intelligently extracting and compressing relevant information, it provides more focused and context-aware responses to queries. This approach has potential applications in various fields requiring efficient and accurate information retrieval from large document collections.

<div style="text-align: center;">

<img src="../images/contextual_compression.svg" alt="contextual compression" style="width:70%; height:auto;">
</div>

### Import libraries

In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from rag_techniques.evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

True

### Define document's path

In [2]:
path = "../data/Understanding_Climate_Change.pdf"

### Create a vector store

In [3]:
vector_store = encode_pdf(path)

### Create a retriever + contexual compressor + combine them 

In [4]:
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
embed_model = CohereEmbeddings(model="embed-english-v3.0")


# Create a retriever
retriever = vector_store.as_retriever()


#Create a contextual compressor
#llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
compressor = LLMChainExtractor.from_llm(llm)

#Combine the retriever with the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Create a QA chain with the compressed retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True
)

### Example usage

In [5]:
query = "What is the main topic of the document?"
result = qa_chain.invoke({"query": query})
print(result["result"])
print("Source documents:", result["source_documents"])

The main topic of the document appears to be climate change and global cooperation to address it, with a focus on international agreements, initiatives, and strategies for mitigation and adaptation.
Source documents: [Document(metadata={'source': '../data/Understanding_Climate_Change.pdf', 'page': 23}, page_content="Kyoto Protocol  \nThe Kyoto Protocol, adopted in 1997, set binding emission reduction targets for developed \ncountries. It was the first major international treaty to address climate change. The protocol \nlaid the groundwork for subsequent agreements, highlighting the importa nce of collective \naction.  \n\nMontreal Protocol  \nThe Montreal Protocol, designed to protect the ozone layer by phasing out ozone -depleting \nsubstances, has also contributed to climate mitigation. The Kigali Amendment to the protocol \ntargets hydrofluorocarbons (HFCs), potent greenhouse gases, demonstrating  the treaty's \nevolving role in climate protection.  \n\nRegional and National Initiat